# 6.6 Vispyによる高パフォーマンス対話型データ可視化
　既存のPython向けグラフ描画やデータ可視化のライブラリは、ほとんどの場合小から中規模のデータセットが対称となっている。ビッグで＾他時代なら、より大規模のデータセットが描画対象となる。
 
　[Vispy](http://vispy.org)は、2Dと3D向けの新しい高パフォーマンスデータ可視化ライブラリである。Vispyは、OpenGLライブラリを通して、現代的なグラフィックプロセッサ(Graphics Processing Unitss:[GPU](https://ja.wikipedia.org/wiki/Graphics_Processing_Unit))の計算力を活用している。
 
　このレシピでは、OpenGLの基礎概念の簡単な説明を行う。ここでは、Vispyのオブジェクト指向OpenGLインターフェースを使って、デジタル信号を表示する。

## 準備
　このレシピは[開発版Vispy](http://github.com/vispy/vispy)を使ってテストしている。GitHubのリポジトリをクローンして、次のコマンドでVispyをインストールする。
 ```python
 python setup.py install
 ```

## 手順
　まず、NumPy、vispy.app(canvasへの描画)、vispy.gloo(OpenGLへのオブジェクト指向インターフェース)をインポートする。

In [12]:
import numpy as np
from vispy import app
from vispy import gloo

### canvas生成
　ウィンドウを表示するために、canvasを生成する必要がある。

In [13]:
c = app.Canvas(keys='interactive')

RuntimeError: context could not be created

### シェーダを作る
　vispy.glooを使う前に、**シェーダ**を作る必要がある。このCに似た言語で作られたプログラムは、GPU上で胴差しデータ可視化に高い柔軟性を提供する。ここでは(a_position変数に格納された)2Dの点を直接canvasに表示する簡単な**頂点シェーダ**を作る。

In [14]:
vertex = """
attribute vec2 a_position;
void main (void)
{
    gl_Position = vec4(a_position, 0.0, 1.0);
}
"""

　**断片シェーダ**も作る。このシェーダは点の色を制御する。ここでは、すべての点を黒で表示。

In [15]:
fragment = """
void main()
{
    gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
}
"""

### OpenGLのプログラム生成
　programオブジェクトに、シェーダと、シェーダへの変数であるNumPyデータが格納される。さらに、1000個の座標を格納した、(1000,2)のNumPy配列をa_position変数にリンクする。デフォルトの座標系では、canvasの4隅は、(1,1)となる。

In [16]:
program = gloo.Program(vertex, fragment)

In [20]:
program['a_position'] = np.c_[np.linspace(-1.0, +1.0, 1000).astype(np.float32),
                              np.random.uniform(-0.5, +0.5, 1000).astype(np.float32)]

### canvasによるコールバック関数の用意
　ウィンドウの大きさが変更された時に呼び出されるコールバック関数と、canvasの表示を更新する必要が生じた際に呼ばれるコールバック関数を用意する。

In [21]:
@c.connect
def on_resize(event):
    gloo.set_viewport(0, 0, *event.size) # OpenGLのviewportを更新して、Vispyがcanvas全体を使えるようにする

NameError: name 'c' is not defined

In [22]:
@c.connect
def on_draw(event):
    gloo.clear((1,1,1,1)) # ウィンドウを白でクリアにする
    program.draw('line_strip') # OpenGLプログラムを使って、線分を描画

NameError: name 'c' is not defined

### 表示
　最後にcanvasを表示してプログラムを実行する。

In [23]:
c.show()
app.run()

NameError: name 'c' is not defined